## Create a Knapsack Problem Instance

In [1]:
from knapsack import Knapsack
from WCGGA import run_WCGGA
# Create an instance of a Knapsack problem
p_file = "Knapsack_Problems/problemInstances/n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300/test.in"
opt_file = "Knapsack_Problems/optima.csv"
knapsack = Knapsack(p_file, opt_file)

n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300
Loaded n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300: 600 items, capacity = 100000000.0, known optim

In [ ]:
run_WCGGA(knapsack, 1000, 0.9, (2/ len(knapsack)), 3600, 100000, 1, 0.75, 42)
# 7573 with Stochastic Sampling for the mother and 5 Tourn group for father
# 7002 with Tourn 2 for father and mother in 20 minutes
# with repair operator on apocolypse
# Crowding = 7713 on n_800_c_1000000_g_6_f_0.1_eps_0.01_s_300
# 99759818 on n_600_c_100000000_g_6_f_0.1_eps_0.01_s_300

Starting Deterministic Crowding Evolution...
Gen 0: P1A=99759413  P1B=99759261  P2A=99759457  P2B=99759119  P3A=99759006  P3B=99759005  P4A=99759200  P4B=99759340  
Gen 1: P1A=99759483  P1B=99759261  P2A=99759457  P2B=99759342  P3A=99759047  P3B=99759410  P4A=99759582  P4B=99759348  
Gen 2: P1A=99759483  P1B=99759570  P2A=99759457  P2B=99759342  P3A=99759259  P3B=99759410  P4A=99759582  P4B=99759684  
Gen 3: P1A=99759483  P1B=99759570  P2A=99759457  P2B=99759480  P3A=99759259  P3B=99759467  P4A=99759582  P4B=99759684  
Gen 4: P1A=99759483  P1B=99759570  P2A=99759506  P2B=99759550  P3A=99759466  P3B=99759467  P4A=99759582  P4B=99759684  
Gen 5: P1A=99759483  P1B=99759570  P2A=99759506  P2B=99759550  P3A=99759468  P3B=99759471  P4A=99759582  P4B=99759684  
Gen 6: P1A=99759629  P1B=99759570  P2A=99759523  P2B=99759550  P3A=99759468  P3B=99759471  P4A=99759659  P4B=99759684  
Gen 7: P1A=99759629  P1B=99759570  P2A=99759523  P2B=99759550  P3A=99759471  P3B=99759471  P4A=99759741  P4B=997597